<a href="https://colab.research.google.com/github/JhoanFuentes/Machine-Learning/blob/main/DETECTOR_DE_TONO_ECON%C3%93MICO_EN_COMUNICADOS_OFICIALES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-

# ==============================================================================
# PROYECTO: DETECTOR DE TONO ECONÓMICO EN COMUNICADOS OFICIALES
# Implementación basada en la exploración del artículo de M. Dell (DL for Economists)
# Compara Fine-tuning de un Transformer vs. Clasificación con GenAI (GPT)
# ==============================================================================

# ------------------------------------------------------------------------------
# **PARTE 0: CONFIGURACIÓN INICIAL E INSTALACIONES**
# ------------------------------------------------------------------------------
print("=== PARTE 0: Configuración Inicial ===")

# Instalación de librerías necesarias en Google Colab
!pip install transformers[torch] datasets evaluate scikit-learn openai pandas -q

import pandas as pd
import numpy as np
import torch
import evaluate  # Librería de Hugging Face para métricas
from datasets import Dataset, DatasetDict, ClassLabel
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from sklearn.metrics import accuracy_score, f1_score
import openai # Para la parte de GenAI
import os
import getpass # Para pedir la API key de forma segura

# Verificar si hay GPU disponible (recomendado para fine-tuning)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")
if device.type == 'cuda':
    print(f"Nombre de la GPU: {torch.cuda.get_device_name(0)}")

print("\nInstalación y carga de librerías completada.")

# ------------------------------------------------------------------------------
# **PARTE 1: PREPARACIÓN DE DATOS (¡SIMULADOS!)**
# ------------------------------------------------------------------------------
print("\n=== PARTE 1: Preparación de Datos (Simulados) ===")

# --- ¡¡¡IMPORTANTE!!! ---
# Estos datos son SIMULADOS y muy pequeños. En un proyecto real, aquí cargarías
# tus datos reales, preprocesados y etiquetados manualmente.
# La calidad y cantidad de tus datos etiquetados es CRUCIAL (Ver Sec 4 del texto).

simulated_data = [
    # --- Optimista ---
    {"text": "La economía muestra signos robustos de recuperación, superando las expectativas del mercado.", "label_text": "Optimista"},
    {"text": "Se proyecta un crecimiento significativo del PIB para el próximo trimestre gracias a la inversión extranjera.", "label_text": "Optimista"},
    {"text": "El consumo interno ha repuntado notablemente, impulsando la actividad comercial.", "label_text": "Optimista"},
    {"text": "Las exportaciones alcanzaron un nuevo récord histórico, fortaleciendo nuestra balanza comercial.", "label_text": "Optimista"},

    # --- Pesimista ---
    {"text": "Persisten presiones inflacionarias que podrían afectar el poder adquisitivo de los hogares.", "label_text": "Pesimista"},
    {"text": "La incertidumbre global sigue representando un riesgo considerable para la inversión local.", "label_text": "Pesimista"},
    {"text": "El mercado laboral aún no recupera los niveles pre-pandemia, mostrando debilidad.", "label_text": "Pesimista"},
    {"text": "Se observa una desaceleración en el sector industrial que requiere atención.", "label_text": "Pesimista"},

    # --- Neutral ---
    {"text": "El comité de política monetaria decidió mantener la tasa de interés de referencia.", "label_text": "Neutral"},
    {"text": "Se publicaron los datos actualizados del índice de precios al consumidor.", "label_text": "Neutral"},
    {"text": "El tipo de cambio fluctuó dentro de los rangos esperados durante la última semana.", "label_text": "Neutral"},
    {"text": "El análisis de los agregados monetarios se presentará en el próximo informe trimestral.", "label_text": "Neutral"},

    # --- Fáctico/Técnico (Podría fusionarse con Neutral o ser una clase aparte) ---
    {"text": "La metodología de cálculo del indicador X se ajustó según la normativa internacional.", "label_text": "Fáctico"},
    {"text": "El modelo econométrico utilizado incorpora variables endógenas y exógenas específicas.", "label_text": "Fáctico"},
    {"text": "Los resultados de la encuesta de opinión empresarial indican una variación interanual del 2.5%.", "label_text": "Fáctico"},
    {"text": "Se aplicó un filtro Hodrick-Prescott para descomponer la serie de tiempo del producto.", "label_text": "Fáctico"},
    {"text": "El coeficiente de Gini se ubicó en 0.52 para el último periodo medido.", "label_text": "Fáctico"},
    {"text": "La deuda pública como porcentaje del PIB se mantuvo estable en el 45%.", "label_text": "Fáctico"},
    {"text": "La inversión fija bruta experimentó un crecimiento trimestral desestacionalizado del 1.2%.", "label_text": "Fáctico"},
    {"text": "El balance fiscal primario presentó un superávit equivalente al 0.8% del PIB.", "label_text": "Fáctico"}
]

# Convertir a DataFrame de Pandas para facilidad
df = pd.DataFrame(simulated_data)

# Mapeo de etiquetas de texto a números (requerido por muchos modelos)
labels = df['label_text'].unique().tolist()
label2id = {label: i for i, label in enumerate(labels)}
id2label = {i: label for i, label in enumerate(labels)}
df['label'] = df['label_text'].map(label2id)

print(f"\nEtiquetas encontradas: {labels}")
print(f"Mapeo label -> id: {label2id}")
print(f"Mapeo id -> label: {id2label}")
print(f"\nNúmero de ejemplos simulados: {len(df)}")
print(f"Distribución de etiquetas simuladas:\n{df['label_text'].value_counts()}")

# Convertir el DataFrame a un Dataset de Hugging Face
hg_dataset = Dataset.from_pandas(df)

# Crear las características (features) incluyendo la ClassLabel
# Esto ayuda a Hugging Face a entender las etiquetas
features = hg_dataset.features
features['label'] = ClassLabel(names=labels)
hg_dataset = hg_dataset.cast(features)

# Dividir el dataset en entrenamiento, validación y prueba (80/10/10 split)
# ¡Con tan pocos datos, esta división es solo ilustrativa! En la práctica,
# necesitas cientos o miles de ejemplos etiquetados.
train_testvalid = hg_dataset.train_test_split(test_size=0.3, seed=42, stratify_by_column="label")
test_valid = train_testvalid['test'].train_test_split(test_size=0.5, seed=42, stratify_by_column="label")

# Crear el DatasetDict final
split_datasets = DatasetDict({
    'train': train_testvalid['train'],
    'validation': test_valid['test'], # Usado para ajustar hiperparámetros y early stopping
    'test': test_valid['train']       # Usado SOLO para la evaluación final
})

print(f"\nDataset dividido:")
print(split_datasets)

# ------------------------------------------------------------------------------
# **PARTE 2: FINE-TUNING DE UN CLASIFICADOR TRANSFORMER**
# ------------------------------------------------------------------------------
print("\n=== PARTE 2: Fine-tuning de Clasificador Transformer ===")
print("Esto puede tardar unos minutos, incluso con GPU y datos pequeños...")

# --- 2.1 Cargar Tokenizer y Modelo Pre-entrenado ---
# Usaremos un modelo BERT pre-entrenado para español.
# 'dccuchile/bert-base-spanish-wwm-uncased' es una opción popular.
model_checkpoint = "dccuchile/bert-base-spanish-wwm-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Cargar el modelo pre-entrenado con una cabeza de clasificación encima.
# Especificamos el número de etiquetas y los mapeos id<->label.
model_finetuned = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id
).to(device) # Mover el modelo a la GPU si está disponible

print(f"\nTokenizer y Modelo '{model_checkpoint}' cargados.")

# --- 2.2 Preprocesamiento (Tokenización) ---
def tokenize_function(examples):
    # Tokeniza los textos. `truncation=True` corta textos largos,
    # `padding=True` (o 'max_length') rellena textos cortos.
    # Max length puede ajustarse según tu análisis de longitud de textos.
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

# Aplicar la tokenización a todos los splits del dataset
tokenized_datasets = split_datasets.map(tokenize_function, batched=True)

# Eliminar columnas innecesarias después de tokenizar
tokenized_datasets = tokenized_datasets.remove_columns(["text", "label_text", "__index_level_0__"])
# Renombrar 'label' a 'labels' que es el nombre esperado por el Trainer
#tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
# Establecer el formato a tensores de PyTorch
tokenized_datasets.set_format("torch")

print("\nDatasets tokenizados y formateados para PyTorch.")
print(tokenized_datasets)

# Data collator se encarga de crear los lotes (batches) de datos
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# --- 2.3 Definir Métricas de Evaluación ---
# Usaremos Accuracy y F1-score (Weighted para manejar posible desbalance)
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_metric.compute(predictions=predictions, references=labels)
    f1_weighted = f1_metric.compute(predictions=predictions, references=labels, average="weighted")
    f1_macro = f1_metric.compute(predictions=predictions, references=labels, average="macro") # Macro es sensible al rendimiento en clases pequeñas
    return {
        "accuracy": acc["accuracy"],
        "f1_weighted": f1_weighted["f1"],
        "f1_macro": f1_macro["f1"],
    }

print("\nFunción para calcular métricas definida.")

# --- 2.4 Configurar Argumentos de Entrenamiento ---
# Estos son hiperparámetros clave. Necesitan ajuste (usando el validation set)
# en un proyecto real. Los valores aquí son solo un ejemplo.
# (Ver Sec 3 sobre optimización en el texto de Dell)
training_args = TrainingArguments(
    output_dir="./results_finetuned",          # Directorio para guardar el modelo y logs
    num_train_epochs=5,                     # Número de épocas (ajustar según convergencia) - pocas por los pocos datos
    per_device_train_batch_size=4,          # Tamaño del lote para entrenamiento (ajustar según memoria GPU)
    per_device_eval_batch_size=8,           # Tamaño del lote para evaluación
    warmup_steps=50,                        # Pasos de calentamiento para el learning rate scheduler
    weight_decay=0.01,                      # Regularización L2
    logging_dir='./logs_finetuned',             # Directorio para logs de TensorBoard
    logging_steps=10,                       # Frecuencia de loggeo
    evaluation_strategy="epoch",            # Evaluar al final de cada época
    save_strategy="epoch",                  # Guardar el modelo al final de cada época
    load_best_model_at_end=True,            # Cargar el mejor modelo encontrado durante el entrenamiento al final
    metric_for_best_model="f1_macro",       # Métrica para decidir cuál es el "mejor" modelo (F1 macro suele ser bueno para clases desbalanceadas)
    greater_is_better=True,                 # La métrica elegida debe maximizarse
    report_to="none",                       # Deshabilitar reportes a WandB/Tensorboard por simplicidad aquí
    fp16=torch.cuda.is_available(),         # Usar precisión mixta si hay GPU (acelera y ahorra memoria)
)

print("\nArgumentos de entrenamiento configurados.")

# --- 2.5 Entrenar el Modelo ---
# Crear el objeto Trainer
trainer = Trainer(
    model=model_finetuned,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"], # ¡Usa validation set para evaluar durante el entrenamiento!
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# ¡Iniciar el entrenamiento!
train_result = trainer.train()

# Guardar métricas de entrenamiento
metrics = train_result.metrics
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state() # Guarda el estado del trainer
trainer.save_model("./results_finetuned/best_model") # Guarda el mejor modelo explícitamente

print("\nEntrenamiento completado.")

# --- 2.6 Evaluar en el Conjunto de Prueba ---
# ¡MUY IMPORTANTE! Evaluar el rendimiento final en el test set que
# el modelo NUNCA vio durante el entrenamiento o la selección de hiperparámetros.
print("\nEvaluando el modelo fine-tuned en el CONJUNTO DE PRUEBA...")
test_metrics = trainer.evaluate(tokenized_datasets["test"])

print("\nMétricas en el Conjunto de Prueba (Fine-tuned):")
for key, value in test_metrics.items():
    # Filtrar métricas relevantes de la evaluación del test set
    if key in ["eval_loss", "eval_accuracy", "eval_f1_weighted", "eval_f1_macro", "eval_runtime", "eval_samples_per_second"]:
        print(f"  {key.replace('eval_', '')}: {value:.4f}")

trainer.log_metrics("test", test_metrics)
trainer.save_metrics("test", test_metrics)


# --- 2.7 Hacer Predicciones con el Modelo Fine-tuned ---
def predict_tone_finetuned(text):
    """Función para predecir el tono de un nuevo texto usando el modelo fine-tuned."""
    # Tokenizar el texto de entrada
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128).to(device)
    # Hacer la predicción (desactivar cálculo de gradientes para inferencia)
    with torch.no_grad():
        logits = model_finetuned(**inputs).logits
    # Obtener la clase predicha (el índice con el logit más alto)
    predicted_class_id = logits.argmax().item()
    # Devolver la etiqueta de texto correspondiente
    return id2label[predicted_class_id]

print("\nProbando predicción con el modelo fine-tuned:")
test_sentence_1 = "El panorama económico parece mejorar lentamente."
test_sentence_2 = "Las cifras fiscales muestran un déficit preocupante."
print(f"Texto: '{test_sentence_1}' -> Tono Predicho: {predict_tone_finetuned(test_sentence_1)}")
print(f"Texto: '{test_sentence_2}' -> Tono Predicho: {predict_tone_finetuned(test_sentence_2)}")


# ------------------------------------------------------------------------------
# **PARTE 3: CLASIFICACIÓN USANDO GENAI (EJEMPLO CON OPENAI GPT)**
# ------------------------------------------------------------------------------
print("\n=== PARTE 3: Clasificación usando GenAI (GPT) ===")

# --- ¡¡¡ADVERTENCIA!!! ---
# Esta parte requiere una API Key de OpenAI.
# El uso de la API de OpenAI tiene COSTOS asociados. Revisa sus precios.
# NUNCA pongas tu API key directamente en el código. Usa Colab Secrets o variables de entorno.

# Intentar obtener la API key de forma segura
try:
    openai_api_key = os.environ.get("OPENAI_API_KEY")
    if not openai_api_key:
         openai_api_key = getpass.getpass("Ingresa tu OpenAI API Key: ")
    openai.api_key = openai_api_key
    # Hacer una llamada de prueba simple para verificar la key (opcional, pero útil)
    client = openai.OpenAI(api_key=openai_api_key)
    client.models.list()
    print("API Key de OpenAI configurada correctamente.")
    use_openai = True
except Exception as e:
    print(f"Error al configurar la API de OpenAI: {e}")
    print("No se pudo configurar la API Key de OpenAI. Saltando la Parte 3.")
    use_openai = False

if use_openai:
    # --- 3.1 Definir la Función de Clasificación con GPT ---
    # El modelo 'gpt-3.5-turbo' es una opción económica y rápida.
    # 'gpt-4' o 'gpt-4o' son más potentes pero más caros.
    # (Ver Sec 7 sobre experiencias de Dell con diferentes modelos GPT/Claude)
    GPT_MODEL = "gpt-3.5-turbo"

    def classify_with_gpt(text_to_classify):
        """Clasifica el texto usando la API de OpenAI con un prompt específico."""
        # Prompt Engineering: Clave para el rendimiento de GenAI (Sec 7)
        # - Ser claro y específico.
        # - Pedir un formato de salida simple (solo la etiqueta).
        # - Usar un conjunto de validación para probar y mejorar prompts (¡no el test set!)
        system_prompt = f"Eres un asistente experto en análisis económico. Tu tarea es clasificar el tono del siguiente texto. Las posibles categorías son: {', '.join(labels)}. Responde ÚNICAMENTE con UNA de esas categorías."
        user_prompt = f"Clasifica el tono del siguiente texto:\n\n\"\"\"\n{text_to_classify}\n\"\"\"\n\nTono:"

        try:
            client = openai.OpenAI(api_key=openai_api_key)
            response = client.chat.completions.create(
                model=GPT_MODEL,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                temperature=0.0, # Baja temperatura para respuestas más deterministas/consistentes
                max_tokens=10,    # Suficiente para una etiqueta corta
                n=1,              # Solo una respuesta
                stop=None         # No se necesitan stops específicos aquí
            )
            # Extraer la respuesta
            predicted_label = response.choices[0].message.content.strip()

            # Validar si la respuesta es una de las etiquetas esperadas
            if predicted_label in labels:
                return predicted_label
            else:
                print(f"  Advertencia: GPT devolvió una etiqueta inesperada: '{predicted_label}'. Se devolverá None.")
                return None # O manejar de otra forma (ej., asignar clase 'Neutral' por defecto)

        except Exception as e:
            print(f"  Error al llamar a la API de OpenAI: {e}")
            return None # Manejo de errores

    print(f"\nFunción para clasificar con {GPT_MODEL} definida.")

    # --- 3.2 Evaluar GenAI en el Conjunto de Prueba ---
    print(f"\nEvaluando {GPT_MODEL} en el CONJUNTO DE PRUEBA...")
    # Usamos el MISMO test set que para el modelo fine-tuned para una comparación justa.
    test_texts_genai = split_datasets["test"]["text"]
    true_labels_genai_text = [id2label[l] for l in split_datasets["test"]["label"]] # Etiquetas verdaderas en formato texto
    predicted_labels_genai = []

    for i, text in enumerate(test_texts_genai):
        print(f"  Procesando ejemplo {i+1}/{len(test_texts_genai)}...", end='\r')
        predicted = classify_with_gpt(text)
        predicted_labels_genai.append(predicted)
        # Pequeña pausa para evitar rate limits (ajustar si es necesario)
        # time.sleep(0.5)
    print("\nProcesamiento GenAI completado.")

    # Filtrar resultados None (donde la API falló o dio formato incorrecto)
    valid_indices = [i for i, pred in enumerate(predicted_labels_genai) if pred is not None]
    valid_true_labels = [true_labels_genai_text[i] for i in valid_indices]
    valid_predicted_labels = [predicted_labels_genai[i] for i in valid_indices]

    if len(valid_predicted_labels) > 0:
        # Calcular métricas para GenAI
        accuracy_genai = accuracy_score(valid_true_labels, valid_predicted_labels)
        # Asegúrate de que las etiquetas usadas en f1_score sean consistentes (todas las posibles o las presentes)
        f1_weighted_genai = f1_score(valid_true_labels, valid_predicted_labels, average="weighted", labels=labels, zero_division=0)
        f1_macro_genai = f1_score(valid_true_labels, valid_predicted_labels, average="macro", labels=labels, zero_division=0)

        print("\nMétricas en el Conjunto de Prueba (GenAI - GPT):")
        print(f"  accuracy: {accuracy_genai:.4f}")
        print(f"  f1_weighted: {f1_weighted_genai:.4f}")
        print(f"  f1_macro: {f1_macro_genai:.4f}")
        print(f"  (Calculado sobre {len(valid_predicted_labels)} de {len(test_texts_genai)} ejemplos válidos)")
    else:
        print("\nNo se pudieron obtener predicciones válidas de GenAI para calcular métricas.")

    # --- 3.3 Probar Predicción con GenAI ---
    print("\nProbando predicción con GenAI:")
    print(f"Texto: '{test_sentence_1}' -> Tono Predicho (GPT): {classify_with_gpt(test_sentence_1)}")
    print(f"Texto: '{test_sentence_2}' -> Tono Predicho (GPT): {classify_with_gpt(test_sentence_2)}")

else:
    print("\nSaltando la Parte 3 debido a problemas con la API Key de OpenAI.")

# ------------------------------------------------------------------------------
# **PARTE 4: COMPARACIÓN Y DISCUSIÓN FINAL**
# ------------------------------------------------------------------------------
print("\n=== PARTE 4: Comparación y Discusión Final ===")

print("\nResumen de Métricas en el Conjunto de Prueba:")
print("---------------------------------------------")
print("| Método                 | Accuracy | F1 Weighted | F1 Macro |")
print("|------------------------|----------|-------------|----------|")
# Imprimir métricas del modelo fine-tuned (ajustando nombres de claves si es necesario)
ft_acc = test_metrics.get('eval_accuracy', float('nan'))
ft_f1w = test_metrics.get('eval_f1_weighted', float('nan'))
ft_f1m = test_metrics.get('eval_f1_macro', float('nan'))
print(f"| Fine-tuned Transformer | {ft_acc:^8.4f} | {ft_f1w:^11.4f} | {ft_f1m:^8.4f} |")

# Imprimir métricas de GenAI si se ejecutó
if use_openai and len(valid_predicted_labels) > 0:
    print(f"| GenAI (GPT-3.5 Turbo)  | {accuracy_genai:^8.4f} | {f1_weighted_genai:^11.4f} | {f1_macro_genai:^8.4f} |")
else:
    print("| GenAI (GPT-3.5 Turbo)  |   N/A    |     N/A     |   N/A    |")
print("---------------------------------------------")

print("\nDiscusión basada en el texto de Dell (Sec 7) y este ejemplo:")
print("1.  **Rendimiento:** En este ejemplo (¡con datos simulados muy pequeños!), los resultados variarán. En la práctica, como señala Dell, un clasificador fine-tuneado con datos de alta calidad específicos del dominio *suele* igualar o superar a GenAI zero-shot/few-shot, especialmente para tareas con matices o domain shift (ej. textos históricos). GenAI puede funcionar bien para tareas directas y dominios cercanos a sus datos de preentrenamiento (web moderno).")
print("2.  **Costo de Desarrollo vs. Inferencia:**")
print("    - **Fine-tuning:** Requiere un esfuerzo inicial mayor (recolección/etiquetado de datos, código de entrenamiento, ajuste de hiperparámetros). Sin embargo, una vez entrenado, la inferencia (hacer predicciones) es muy barata y rápida, especialmente con modelos base eficientes (como DistilBERT o ALBERT).")
print("    - **GenAI:** El costo inicial de desarrollo es bajo (principalmente prompt engineering). Sin embargo, el costo de inferencia puede ser significativo si se procesan grandes volúmenes de texto, ya que cada clasificación requiere una llamada a la API (potencialmente costosa). Los costos y la latencia dependen del proveedor y modelo.")
print("3.  **Control y Reproducibilidad:**")
print("    - **Fine-tuning:** Ofrece mayor control. Puedes inspeccionar el modelo, reentrenarlo con datos específicos para corregir errores, y el modelo entrenado es tuyo, asegurando reproducibilidad si guardas el modelo y el código.")
print("    - **GenAI:** Es más una 'caja negra'. Dependes de la API del proveedor, que puede cambiar sin previo aviso, afectando la reproducibilidad y el rendimiento. Tienes menos control directo sobre el comportamiento del modelo más allá del prompt.")
print("4.  **Facilidad de Uso:**")
print("    - **Fine-tuning:** Requiere conocimientos técnicos de ML/DL (aunque librerías como Hugging Face simplifican mucho).")
print("    - **GenAI:** Más accesible para usuarios sin experiencia en ML, la barrera principal es el prompt engineering y el manejo de la API.")
print("5.  **Adaptabilidad al Dominio:** Fine-tuning permite adaptar el modelo específicamente a tu dominio (ej. lenguaje económico particular, textos históricos), lo cual es crucial si difiere mucho de los datos de preentrenamiento de GenAI (Sec 4, Domain Shift).")

print("\n**Conclusión del Ejemplo:** Ambos enfoques tienen ventajas y desventajas. La elección depende de los recursos disponibles (tiempo, presupuesto, datos etiquetados, experiencia técnica), la escala del problema, los requisitos de rendimiento y la necesidad de control/reproducibilidad, como se discute en la Figura 1 del artículo de Dell.")
print("\n**¡RECORDATORIO FINAL!:** Este fue un ejemplo con datos simulados. ¡Un proyecto real requiere un esfuerzo considerable en la obtención y etiquetado de datos de calidad!")